<a href="https://colab.research.google.com/github/rubyvanrooyen/katcomm/blob/master/Users/ruby/oh_masers/G330/Caracal_standard_WB_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Update libraries

For Python 3.7+, dicts preserve insertion order. Since PyYAML 5.1.x, you can disable the sorting of keys.    
Unfortunately, the sorting keys behaviour does still default to `True`.

In [1]:
import yaml
yaml.__version__

'3.13'

If version <`5.1.x`

In [2]:
!pip install pip --upgrade

     |████████████████████████████████| 1.6 MB 7.6 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [3]:
!pip install pyyaml --upgrade

     |████████████████████████████████| 636 kB 9.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import yaml
yaml.__version__

'5.4.1'

# G330 input
```
Targets: 3 selected out of 3 in catalogue
  ID  Name          Type      RA(J2000)     DEC(J2000)  Tags
   0  J1939-6342    radec     19:39:25.03  -63:42:45.6  fluxcal bpcal delaycal 
   1  G330.89-0.36  radec     16:10:20.54  -52:06:14.9  target  
   2  J1726-5529    radec     17:26:49.63  -55:29:40.5  gaincal
```

In [15]:
import os

msfile = '1625501775_sdp_l0.ms'
name = 'G330.89-0.36'
prefix, _ = os.path.splitext(msfile)

config = f'config-{prefix}-4k.yml'
output = f'output-{prefix}-4k'

b_cal = 'J1939-6342'
f_cal = 'J1939-6342'
g_cal = 'J1726-5529'
target = 'G330.89-0.36'

ref_ant = ''
ref_chan = ''

# Stimela workers
Construct skeleton stimela workers

### Compulsary workers
* `general`
Compulsory worker to set up data/input/output directories.
* `getdata`
Compulsory worker to specify the input .MS files.
* `obsconf`
Compulsory worker to set up target/calibrators names.

In [16]:
# The prefix used for the output data products (e.g., diagnostic plots, images, etc.).
worker_general = {'title': name,
                  'prefix': prefix,
                  'rawdatadir': 'data',
                  'output': output,
                  }

In [17]:
worker_getdata = {'dataid': f'[{os.path.splitext(msfile)[0]}]',
                  'extension': 'ms',
                  }

In [18]:
worker_obsconf = {'refant': ref_and if ref_ant else 'auto',
                  }

### Calibrators
* `transform`
Split calibrators-only .MS files, one per input .MS file.
* `prep`
Prepare the calibrators-only .MS files for processing.
* `flag`
Flag the calibrators-only .MS files.
* `crosscal`
Derive the cross-calibration tables and apply them to the calibrators.
* `inspect`
Inspect the calibrated calibrator’s visibilities to check the quality of the
cross-calibration.

In [19]:
# This worker splits the calibrators (in preparation for cross-calibration) or the targets (in
# preparation for imaging) to new .MS files. Time and frequency averaging is available, as well as
# phase rotation to a new phase centre. Crosscalibration can be applied on the fly while splitting.

# https://caracal.readthedocs.io/en/latest/manual/workers/transform/index.html#split-field
split_field = {'enable': "true",
               'col': 'data',
               }
# https://caracal.readthedocs.io/en/latest/manual/workers/transform/index.html#
worker_transform__calibrators = {'enable': 'true',
                                 'label_out': 'cal',
                                 'field': 'calibrators',
                                 'split_field': split_field,
                                 }

In [20]:
# https://caracal.readthedocs.io/en/latest/manual/workers/prep/index.html#specweights
specweights = {'enable': 'true',
               'mode': 'uniform',
               }
# https://caracal.readthedocs.io/en/latest/manual/workers/prep/index.html
worker_prep__calibrators = {'enable': 'true',
                            'label_in': 'cal',
                            'field': 'calibrators',
                            'specweights': specweights,
                            }

#### Flagging

Standard flags to apply to all data sets


In [21]:
flag_elevation = {'enable': 'true',
                  'low': '15',
                  'high': '90',
                  }
flag_spw = {'enable': 'true',
            'chans': '*:856~880MHz , *:1658~1800MHz, *:1419.8~1421.3MHz',
            'ensure_valid': 'false',
            }

# GSM and Aviation
# *:900MHz~915MHZ,*:925MHz~960MHZ,
# Satellites
# *:1080MHz~1095MHZ,*:1166MHz~1186MHZ,*:1191MHz~1237MHZ,*:1242MHz~1249MHZ,*:1260MHz~1300MHZ,*:1375MHz~1387MHZ,
# *:1526MHz~1554MHZ,*:1565MHz~1585MHZ,*:1592MHz~1610MHZ,*:1616MHz~1626MHZ,
flag_mask = {'enable': 'true',
             'mask': 'meerkat.rfimask.npy',  # np.load('caracal/data/meerkat_files/meerkat.rfimask.npy')
             'uvrange': '0~1000',
             }

# https://caracal.readthedocs.io/en/latest/manual/workers/flag/index.html#
worker_flag__calibrators = {'enable': 'true',
                            'label_in': 'cal',
                            'field': 'calibrators',
                            # flag auto-correlations
                            'flag_autocorr': {'enable': 'true'},
                            # remove any antennas shadowing each other
                            'flag_shadow': {'enable': 'true'},
                            # remove points below minimum pointing angle of observation
                            'flag_elevation': flag_elevation,
                            # flag bandpass edges and the Milky Way
                            'flag_spw': flag_spw,
                            # GSM, aviation and satellites
                            'flag_mask': flag_mask,
                            }

Flag RFI using AOFlagger, Tricolour, or CASA FLAGDATA with tfcrop.

In [22]:
# https://caracal.readthedocs.io/en/latest/manual/workers/flag/index.html#flag-rfi
worker_flag__rfi = {'enable': 'true',
                    'field': 'calibrators',
                    'label_in': 'cal',
                    'flag_rfi': {'enable': 'true',
                                 'flagger': 'tricolour',
                                 'tricolour': {'strategy': 'khruschev.yaml'},
                                 }
                    }

Data for manual flagging can be identified either using CASA or shadeMS (WIP)
* CASA: https://github.com/rubyvanrooyen/katcomm/blob/master/Users/ruby/oh_masers/G330/G330_CASA_data_inspection_commands.ipynb
* shadeMS: <link to notebook>

In [23]:
flag_spw = {'enable': 'true',
            'chans': '*:*:1.1421GHz~1.16592GHZ,*:1.1862GHz~1.25997GHZ,*:1.52057GHz~1.59185GHZ,*:1.62632GHz',
            'ensure_valid': 'false',
            }

#flag_manual = {'enable': 'true',
#               'rules': 
#               }
##   flag_manual:
##     enable: true
##     rules:
##       - '1557* antenna:m007'
##      - '1557* scan:49'
##      - '1568* antenna:m024;m036'
##      - '1568* scan:35 uvrange:1000~9000'

worker_flag__manual = {'enable': 'true',
                       'label_in': 'cal',
                       'field': 'calibrators',
                       # flag out manually identified RFI channels
                       'flag_spw': flag_spw,
                       }

#### Inspect
Quick view of data after flagging


In [ ]:
worker_inspect = {enable: true,
                  label_in: 'cal',
                  field: 'calibrators',
                  label_plot: 'flags',
                  dirname: flag,
                  correlation: all,
                  standard_plotter: none,

  shadems = {enable: true,
             default_column: DATA,
             plots:
      # per-antenna plots
      - iter_ant:
        desc: "plots by antenna",
        cmap: pride,
        corr: XX,YY,
        plots:
          - '-x CHAN -y amp -c CORR --field {bpcal} --dir bpcal-iamp-byant-{msbase}'
    ignore_errors: true

In [ ]:
# https://caracal.readthedocs.io/en/latest/manual/workers/inspect/index.html
inspect:
  enable: true
  label_in: 'cal'
  field: 'calibrators'
  label_plot: 'flags'
  dirname: flag
  correlation: all
  standard_plotter: none
  shadems:
    enable: true
    default_column: DATA
    plots:
      # per-antenna plots
      - iter_ant:
        desc: "plots by antenna"
        cmap: pride
        corr: XX,YY
        plots:
          - '-x CHAN -y amp -c CORR --field {bpcal} --dir bpcal-iamp-byant-{msbase}'
    ignore_errors: true


# Recommended workflow

https://caracal.readthedocs.io/en/latest/manual/reduction/workflow/index.html

```
$ caracal --version
caracal version 1.0.4
```

In [24]:
std_caracal_dict = {'schema_version': '1.0.4',
                    # Compulsory workers
                    'general': worker_general,
                    'getdata': worker_getdata,
                    'obsconf': worker_obsconf,
                    # Calibration
                    'transform__calibrators': worker_transform__calibrators,
                    'prep__calibrators': worker_prep__calibrators,
                    'flag__calibrators': worker_flag__calibrators,
                    'flag__rfi': worker_flag__rfi,
                    'flag__manual': worker_flag__manual,
                    }
# print(std_caracal_dict)

In [25]:
import yaml

with open(config, 'w') as outfile:
    yaml.dump(std_caracal_dict, outfile, default_flow_style=False, sort_keys=False, default_style=None)

print(f'created file {config}')

created file config-1625501775_sdp_l0-4k.yml


In [26]:
!sed "s/'//g" config-1625501775_sdp_l0-4k.yml > tmpfile.yaml
!mv tmpfile.yaml config-1625501775_sdp_l0-4k.yml

In [27]:



# Derive the cross-calibration tables and apply them to the calibrators.
# Carry out Cross calibration of the data (delay, bandpass and gain calibration).
crosscal:
  enable: true
  uvrange: '>150'
  label_in: 'cal'
  label_cal: '1gc1'
  set_model:
    enable: true
    meerkat_skymodel: false
  primary:
    reuse_existing_gains: true
    order: KGBAKGB
    combine: ["", "", "", null, "", "", ""]
    solint: [inf, inf, inf, null, int, int, inf]
    calmode: [a, ap, ap, null, a, ap, ap]
    b_fillgaps: 70
    plotgains: true
  secondary:
    reuse_existing_gains: true
    order: KGAKF
    apply: B
    combine: ["", "", null, "", ""]
    solint: [inf, inf, null, 60s, 60s]
    calmode: [a, ap, null, a, ap]
    plotgains: true
  apply_cal:
    applyto:
      - gcal
      - bpcal

# This worker plot the visibilities for diagnostic purpose.
# Inspect the calibrated calibrator’s visibilities to check the quality of the
# cross-calibration.
inspect:
  enable: true
  label_in: 'cal'
  field: 'calibrators'
  label_plot: '1gc'
  dirname: crosscal
  correlation: all
  standard_plotter: none # ragavi_vis
  shadems:
    enable: true
    default_column: CORRECTED_DATA
    plots:
      - desc: "plots by field"
        field: "{all_fields}"
        cnum: 100  # up 100 colours
        iter_field: true
        plots:
          # phaseball plots
          - dir: "phaseballs-{msbase}"
            plots:
              - "-x real -y imag -c CORR --corr IQUV --hline 0: --vline 0:"
              - "-x real -y imag -c SCAN_NUMBER"
              - "-x real -y imag -c ANTENNA1"
          - dir: "phaseballs-bycorr-{msbase}"
            iter_corr:
            plots:
              - "-x real -y imag -c SCAN_NUMBER"
              - "-x real -y imag -c ANTENNA1"
          # normalized phaseballs
          - dir: "normballs-{msbase}"
            col: "CORRECTED_DATA/MODEL_DATA"
            corr: "XX,YY"
            iter_corr:
            plots:
              - "-x real -y imag -c SCAN_NUMBER"
              - "-x real -y imag -c ANTENNA1"
          # block and triangle plots
          - dir: "blockplots-{msbase}"
            plots:
              - "-x BASELINE_M -y FREQ -c amp"
              - "-x ANTENNA1 -y ANTENNA2 -c SCAN_NUMBER --aaxis phase --ared std"
              - "-x ANTENNA1 -y ANTENNA2 -c SCAN_NUMBER --aaxis amp --ared mean"
          # amp/phase versus uv-distance, and uv-coverage coloured by amp/phase
          - dir: "uvdist-{msbase}"
            plots:
              - "-x UV -y amp    -c SCAN_NUMBER"
              - "-x UV -y amp    -c ANTENNA1"
              - "-x UV -y phase  -c ANTENNA1 --corr XX,YY"
              - "-x U  -y V      -c amp"
              - "-x U  -y V      -c phase --cmin -5 --cmax 5"
          # spectral plots
          - dir: "spectra-{msbase}"
            plots:
              - "-x FREQ  -y amp  -c SCAN_NUMBER"
              - "-x FREQ  -y amp  -c ANTENNA1"
              - "-x FREQ  -y real -c CORR --corr IQUV --hline 0:"
      # per-antenna plots
      - iter_ant:
        desc: "plots by antenna"
        cmap: pride
        corr: XX,YY
        plots:
          - '-x FREQ -y amp:I -c SCAN_NUMBER --cnum 100 --cmin 0 --cmax 30 --field {bpcal} --dir bpcal-iamp-byant-{msbase}'
          - '-x FREQ -y SCAN_NUMBER -a amp --ared std --dir stdamp-byant-{msbase}'
          - '-x FREQ -y SCAN_NUMBER -a phase --ared std --dir stdphase-byant-{msbase}'
      # per-scan plots
      - iter_scan:
        desc: "plots by scan"
        cmap: pride
        ared: std
        corr: XX,YY
        plots:
          - '-x ANTENNA1 -y ANTENNA2 -a amp --dir stdamp-byscan-{msbase}'
          - '-x ANTENNA1 -y ANTENNA2 -a phase --dir stdphase-byscan-{msbase}'
          - '-x BASELINE_M -y FREQ -a imag --amin 0 --amax 2 --dir stdimag-byscan-{msbase}'
    ignore_errors: true


SyntaxError: ignored

## Targets

In [ ]:
# Split target-only .MS files, one per input .MS file and target, applying the
# cross-calibration on the fly.
transform__target:
  enable: true
  label_out: &CORR corr
  split_field:
    enable: true
    chan_avg: 4
    otfcal:
      enable: true
      label_cal: '1gc1'
      interpolation:
        delay_cal: nearest
        gain_cal: linear


In [ ]:





# Prepare the target-only .MS files for processing.
prep__target:
  enable: true
  label_in: *CORR
  field: target
  specweights:
    enable: true
    mode: uniform

# Flag the target-only .MS files.
flag__target:
  enable: true
  field: target
  label_in: *CORR
  flag_spw:
    enable: true
    chans: '*:856~880MHz , *:1658~1800MHz, *:1419.8~1421.3MHz'
#    chans: '*:856~940MHz, *:1180~1300MHz, *:1658~1800MHz, *:1419.8~1421.3MHz'
    ensure_valid: false
  flag_mask:
    enable: true
    mask: meerkat.rfimask.npy
    uvrange: '0~1000'
#   flag_manual:
#     enable: true
#     rules:
#       - '1557* antenna:m007'
#       - '1568* antenna:m024;m036'
  inspect:
    enable: false
    field: target

flag__target_rfi:
  enable: true
  field: target
  label_in: *CORR
  flag_rfi:
    enable: true
    flagger: tricolour
    tricolour:
      strategy: khruschev.yaml

# This worker creates an a-priori clean mask based on NVSS or SUMSS, to be used during the continuum
# imaging/self-calibration loop.
mask:
  enable: false

# This worker performs continuum imaging and standard (i.e., direction-independent) self-calibration.
# Make a continuum image of each target, self-calibrate, and transfer both gains
# and continuum model to the full-frequency-resolution target-only .MS files.
selfcal:
  enable: true
  img_npix: 10000
  img_cell: 0.8
  img_nchans: 8
  img_multiscale: false
  img_niter: 100000
  img_specfit_nrcoeff: 4
  cal_timeslots_chunk: 12
  image:
    enable: true
  calibrate:
    enable: true
    gsols_timeslots: [4]
    gsols_chan: [256]
    gain_matrix_type: [GainDiagPhase, GainDiagPhase]
  cal_cubical:
    dist_max_chunks: 12
  transfer_model:
    enable: false

# Perform direction-dependent calibration on the data
ddcal:
  enable: false
  label_in: 'corr'
  use_pb: true
  calibrate_dd:
    enable: True
    de_sources_mode: manual
  copy_data:
    enable: true
  image_wsclean:
    enable: true
  transfer_model_dd:
    enable: true


# -fin-
